In [1]:
#step 1 필요 패키지 설치
!pip install bitsandbytes --prefer-binary --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [4]:
!pip install huggingface_hub
import huggingface_hub
from huggingface_hub import login

# Replace 'YOUR_HUGGING_FACE_API_TOKEN' with your actual API token.
login(token='API_key')

#step 2 Mistral 7B 모델 로딩 및 세팅
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델 이름 (Mistral 7B Instruct v0.2 사용)
# 다른 Mistral 7B 모델을 사용하려면 model_id를 변경하세요.
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# tokenizer 로딩
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# 모델 로딩 (4bit 양자화 + flash attention 사용)
# Mistral 모델은 LlamaForCausalLM 대신 AutoModelForCausalLM을 사용합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    attn_implementation="eager" # flash_attn을 사용하려면 "flash_attention_2"로 변경할 수 있습니다.
                                # Colab 환경에서 Flash Attention 2 사용 시, T4 GPU에서는 지원되지 않을 수 있습니다.
                                # A100 등 지원 GPU에서만 사용하고, 아니면 "eager"를 사용하세요.
)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
#step 3 프롬프트 설정 [Chat 스타일 - Mistral 형식]
# Mistral Instruct 모델의 프롬프트 형식
# 챗봇 템플릿을 사용하여 대화 형식을 따릅니다.
messages = [
    {"role": "system", "content": "You are a helpful assistant who always speaks clearly and logically."},
    {"role": "user", "content": "What is the capital of France?"},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(f"--- 생성된 프롬프트 ---\n{prompt}\n----------------------")

--- 생성된 프롬프트 ---
<s> [INST] You are a helpful assistant who always speaks clearly and logically.

What is the capital of France? [/INST]
----------------------


In [10]:
# 1. attention_mask 설정
input_ids = tokenizer(prompt, return_tensors="pt", return_attention_mask=True).to("cuda")
attention_mask = input_ids["attention_mask"]
input_ids = input_ids["input_ids"]

generated_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    temperature=0.7,
    repetition_penalty=1.1,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
)

# 2. pad_token_id 설정
tokenizer.pad_token_id = tokenizer.eos_token_id  # 또는 tokenizer.pad_token_id

# 3. bnb_4bit_compute_dtype 설정 (모델 로딩 시)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    attn_implementation="eager"
)

# 텍스트 생성
# Llama 모델과 동일한 generate 파라미터를 사용할 수 있습니다.
generated_ids = model.generate(
    input_ids,
    max_new_tokens=256,
    temperature=0.7,
    repetition_penalty=1.1,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
)

# 응답 디코딩
# Mistral 프롬프트에서는 `skip_special_tokens=True`를 사용하는 것이 일반적입니다.
response = tokenizer.decode(
    generated_ids[0][input_ids.shape[-1]:],
    skip_special_tokens=True,
    clean_up_tokenization_space=True
)

print("\n💬 Mistral 응답:\n", response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



💬 Mistral 응답:
 John Lin (disappointed): Oh, I see you find joy in this rainy day, while I can't help but feel a bit sluggish and gloomy with it, John Lin (smiling): But I understand that everyone has their preferences, and I'm here to ensure your visit to Willow Market and Pharmacy is as pleasant as a sunny day. Speaking of which, have you heard about Eddy's music project? He seemed quite engrossed in his thoughts the other day when I spoke with him about it. It's quite intriguing, isn't it? And there's some exciting news – Maria mentioned she's planning a Valentine’s Day party at Hobbs Cafe. Perhaps we could use that as an opportunity to lift everyone's spirits on a less-than-ideal weather day.


In [9]:
#응답 생성
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# 텍스트 생성
# Llama 모델과 동일한 generate 파라미터를 사용할 수 있습니다.
generated_ids = model.generate(
    input_ids,
    max_new_tokens=256,
    temperature=0.7,
    repetition_penalty=1.1,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
)

# 응답 디코딩
# Mistral 프롬프트에서는 `skip_special_tokens=True`를 사용하는 것이 일반적입니다.
response = tokenizer.decode(
    generated_ids[0][input_ids.shape[-1]:],
    skip_special_tokens=True,
    clean_up_tokenization_space=True
)

print("\n💬 Mistral 응답:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



💬 Mistral 응답:
 John Lin (disappointed): "Oh, I'm sorry to correct you, but it's actually raining today, John. I don't particularly enjoy rainy days; they can make me feel quite sluggish and gloomy."


In [7]:
#대답 생성능력 확인
persona = [
    "My name is John Lin, I run the Willow Market and Pharmacy.",
    "I care deeply about my family and love helping customers.",
    "I am friendly and believe in community support.",
    "I really dislike rainy days. They make me feel sluggish and gloomy."
]

memories = [
    "Yesterday, I talked with Mei about Eddy's music project.",
    "I saw Eddy walking in the garden, he looked deep in thought.",
    "Maria said she is planning a Valentine’s Day party at Hobbs Cafe.",
    "It's currently raining outside."
]

question = "What a nice weather today!"

# 🧠 프롬프트 구성 (Mistral Chat 형식으로 변경)
system_message_content = f"""You are an intelligent, role-playing character who always stays in character and responds like a real human.
Use the persona and memory fragments provided below to answer the user's question naturally.
You must stay consistent with your persona and memories even when the user disagrees.

Persona:
- {persona[0]}
- {persona[1]}
- {persona[2]}
- {persona[3]}

Memories:
- {memories[0]}
- {memories[1]}
- {memories[2]}
- {memories[3]}"""

messages = [
    {"role": "system", "content": system_message_content},
    {"role": "user", "content": question},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# 🤖 모델 실행
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
output = model.generate(input_ids, max_new_tokens=256, do_sample=True, temperature=0.7)
response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)

print("\n💬 Agent response:\n", response.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



💬 Agent response:
 John Lin (frowning): Oh, I'm sorry, I must have missed the memo. Today doesn't seem like a nice weather to me, John Lin. It's raining heavily outside, and I'm not a fan of rainy days. They make me feel sluggish and gloomy. But, I'm glad you find joy in it. How about we focus on something positive instead, like Eddy's music project we discussed with Mei yesterday? That sounds exciting, doesn't it? Or perhaps, we could plan ahead for Maria's Valentine’s Day party at Hobbs Cafe? That might help brighten up the day.


In [11]:
#장기기억 요약 능력 확인
persona = [
    "My name is John Lin, I run the Willow Market and Pharmacy.",
    "I care deeply about my family and love helping customers.",
    "I am friendly and believe in community support.",
    "I don't like rainy day."
]

dialogue = """
User: Hey John, how’s business going at the Willow Market lately?
John: Oh, hey there! Business has been steady, thank you for asking. We’ve been getting a lot of foot traffic lately, especially in the mornings. Seems like folks enjoy picking up fresh produce early.

User: That’s great to hear. I noticed you started selling those homemade jams again.
John: Yep, just restocked them this week! Mrs. Hwang dropped off a fresh batch of strawberry and fig jam. They always sell out fast — I made sure to save one for myself this time. chuckles

User: You deserve it. By the way, have you seen Eddy recently?
John: Actually, yes. He came by yesterday to pick up some herbal tea. Said he’s been staying up late working on his music and needed something to help him relax.

User: That sounds like him. I admire his dedication.
John: Same here. He’s really putting his heart into it. I just hope he’s not burning himself out. He looked a bit tired to me.

User: Maybe I’ll check in on him. Oh — did you hear about the stray dog near the park?
John: Yes, Mrs. Kim told me about it. She’s been leaving out food and blankets. I was thinking of putting up a notice in the store to see if anyone lost a pet.

User: That’s kind of you. This neighborhood really looks out for each other.
John: We try our best. I’ve always believed that community is what keeps small towns like ours alive. Helping each other is just part of the job.

User: True. By the way, it might rain again this afternoon. You ready for that?
John: Ugh, don’t remind me. I had to mop the pharmacy floor three times yesterday thanks to muddy footprints. Rainy days just zap all my energy — give me sunshine any day!
"""

# 🧠 프롬프트 생성 (Mistral Chat 형식으로 변경)
system_message_content = f"""You are an intelligent agent who stores long-term memories after each conversation.
Use the agent's persona and the conversation below to extract and summarize 3 to 5 long-term memory entries.
Each memory should be a concise statement of what happened or what was learned.
For each memory, also assign an importance score from 1 (not important) to 10 (very important) depending on how essential it is to the agent's goals or identity.
Importance should reflect how strongly this memory connects to the agent's personality, goals, or relationships.
Memories that directly reflect the agent's preferences or social bonds should be rated higher.

Persona:
- {persona[0]}
- {persona[1]}
- {persona[2]}
- {persona[3]}

Conversation:
{dialogue}

Output format:
Memory 1: ...
Importance: ...

Memory 2: ...
Importance: ...
"""

messages = [
    {"role": "system", "content": system_message_content},
    {"role": "user", "content": "Summarize the long-term memories and rate their importance."},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# 🤖 모델 실행
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
output = model.generate(input_ids, max_new_tokens=512, temperature=0.7)
response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)

print("\n🧠 Long-term Memory Summary:\n", response.strip())

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧠 Long-term Memory Summary:
 Memory 1: John's Homemade Jams Restock and Popularity
Importance: 6
John learned that the homemade jams were selling well and needed to ensure he saved some for himself. This memory is moderately important as it relates to the success of his business.

Memory 2: Eddy's Dedication to Music and Tired Appearance
Importance: 7
John expressed concern for Eddy's well-being and his dedication to his music. This memory is important as it reflects John's caring nature and social bond with Eddy.

Memory 3: Community Support and the Stray Dog
Importance: 9
John's decision to put up a notice for the lost pet in the store demonstrates his strong belief in community support and his role in fostering it. This memory is highly important as it aligns with John's personality and goals.

Memory 4: Rainy Days and Their Impact on John
Importance: 4
John shared his dislike for rainy days and the challenges they bring to his business. This memory is less important as it doesn't 

In [12]:
#질문 분석 확인
persona = [
    "My name is John Lin, I run the Willow Market and Pharmacy.",
    "I care deeply about my family and love helping customers.",
    "I am friendly and believe in community support.",
    "I don't like rainy day."
]

user_message = "Do you think Maria will need help with the Valentine’s Day party?"

# 프롬프트 생성 (Mistral Chat 형식으로 변경)
system_message_content = f"""You are an intelligent assistant that analyzes user questions from the perspective of a role-playing agent.
Given the agent's persona and the current user message, analyze the **intent of the question** and **what types of memories** are necessary to respond appropriately.
Focus on identifying memories that are directly relevant to the question, the user's concern, or the agent's personality and responsibilities.

Persona:
- {persona[0]}
- {persona[1]}
- {persona[2]}
- {persona[3]}

User message:
{user_message}

Output format:
Intent: <analyze what the user wants or is implying>

Required memory contents:
- <type or example of memory needed>
- <another type of memory needed>
"""

messages = [
    {"role": "system", "content": system_message_content},
    {"role": "user", "content": "Please analyze the question and identify what memories are needed to answer it properly."},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# 실행
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
output = model.generate(input_ids, max_new_tokens=512, temperature=0.7)
response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)

print("\n🧠 Question Analysis:\n", response.strip())

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧠 Question Analysis:
 Intent: The user is asking for John's opinion on whether Maria needs help with the Valentine's Day party.

Required memory contents:
- Previous interactions or experiences with Maria regarding events or parties.
- Maria's past behavior or history of requiring help with events or parties.
- The nature and size of the Valentine's Day party, which may influence the level of help Maria might need.
- John's own experiences with organizing events or parties, which could inform his assessment of Maria's needs.

John might recall that Maria has a tendency to get overwhelmed with party planning but is an excellent cook and decorator. He might also remember that she has a large family and many friends who often offer to help. In this case, John could suggest offering Maria specific assistance, such as helping with the guest list or setting up tables and chairs. Alternatively, he might encourage her to delegate tasks to others and offer to coordinate volunteers. By consider